<a href="https://colab.research.google.com/github/YounSooKimTech/self_study/blob/main/TestData_Preprocessing_Lan_detect_and_translate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

df_test = pd.read_excel("/content/SERVICES_RISKS.xlsx")
df_test['RISK_DESCRIPTION'].fillna("", inplace=True)
df_test["Standard_risk_L2"].fillna("", inplace=True)

df_test.head(3)

,COUNTRY,Max of REVIEW_YEAR,RISK_DESCRIPTION,Standard_risk_L2,Standard_risk_L1,LIKEHOOD,IMPACT,OVERALL_RISK,RISK_TREATMENT,MITIGATION_MEASURES,COMMON_SERVICE_LINE,COMMON_SERVICE_CATEGORY,COMMON_SERVICE
0,Afghanistan,2023,A funding cut imposed by the legislative bodies,,NaN,Possible,Significant Concern,High,Accept,Adjust programme delivery according to the fun...,Common Administration services and including c...,Premises management,Management of common premises
1,Afghanistan,2023,A funding cut imposed by the legislative bodies,,NaN,Possible,Significant Concern,High,Accept,Adjust programme delivery according to the fun...,Common Administration services and including c...,Premises management,Management of common premises
2,Afghanistan,2023,A funding cut imposed by the legislative bodies.,,NaN,Possible,Significant Concern,High,Accept,Adjust programme delivery according to the fun...,Common Administration services and including c...,Premises management,Management of common premises


In [3]:
! pip install langdetect
! pip install deep_translator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 12.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993224 sha256=819493ca56e9fa10a34c071ec0c73e0b877487c09b255f3f1b2404f1f5233125
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 836.7 kB/s eta 0:00:00


In [4]:
# detect languages by using langdetect
# code for langs: # en = "english", es = "espanol", fr = "french"
# https://en.wikipedia.org/wiki/List_of_ISO_639-1_codes


from langdetect import detect

def detect_language(text):
    try:
        return detect(text)
    except:
        return "unknown"

df_test["Language"] = df_test["RISK_DESCRIPTION"].apply(detect_language)

In [5]:
# proportion of each languages
round(df_test["Language"].value_counts()/ len(df_test), 3)

en         0.849
es         0.068
fr         0.048
no         0.013
it         0.008
de         0.005
tl         0.003
ca         0.002
da         0.002
af         0.001
unknown    0.001
Name: Language, dtype: float64

In [6]:
# this code cells shows the langauge categorized other than 3 major langs
# by seeing the discription, we can see whether it is correctly categorized or not

lang_list = df_test["Language"].value_counts().index.tolist()
major_lang = ["en", "es", "fr"]

for a_lang in lang_list:
  if a_lang not in major_lang:
    subset_df = df_test.loc[df_test["Language"] == a_lang, ["RISK_DESCRIPTION", "Language"]]
    print(subset_df, "\n")

    RISK_DESCRIPTION Language
446     Port strikes       no
653             None       no
654             None       no
655             None       no
656             None       no
657             None       no
659             None       no
660             None       no
661             None       no
662             None       no
663             None       no
664             None       no 

                                      RISK_DESCRIPTION Language
140                              situation securitaire       it
249                              Sole service provider       it
295  ICT - UNCP datacenter: damage to data center o...       it
492  market decline due to economic challenges, sou...       it
658                                               None       it
701                 Closure due to financial stability       it
852  Entities cannot participate in common contract...       it
922  Service disruption due to natural disasters. A...       it 

                              

In [7]:
# Based on the print, I figured out as follows:
# None was miscategorized as "no"
# Unknown can be deleted since 123 was used for testing
# it(Italian), da(Danish), ro(Romanian), tl(Tagalong), ca(Catalan) were miscategorized

In [8]:
# correct "no" to english

df_test.loc[df_test["Language"] == "no", "Language"] = "en"


In [9]:
# delete the unknown

df_test = df_test[df_test["Language"] != "unknown"]

In [10]:
# change other languages which were misclassified into English

lang_list = df_test["Language"].value_counts().index.tolist()
major_lang = ["en", "es", "fr"]

lang_mistake = [lang for lang in lang_list if lang not in major_lang and lang != "unknown"]
lang_mistake

for a_lang in lang_mistake:
  df_test.loc[df_test["Language"] == a_lang, "Language"] = "en"


In [11]:
# languages are correctly classified into 3 categories

round(df_test["Language"].value_counts()/ len(df_test), 3)

en    0.885
es    0.068
fr    0.048
Name: Language, dtype: float64

In [12]:
# translate the text into English

from deep_translator import GoogleTranslator

def translate_in_english(text):
  translated = GoogleTranslator(source="auto", target="en").translate(text)
  return translated

df_test["translated"] = df_test["RISK_DESCRIPTION"].apply(translate_in_english)

In [13]:
# tokenization, lemmatization, remove punctuations and numbers


import spacy
nlp = spacy.load("en_core_web_sm")

def tokenize_lemma_and_remove_stopwords_and_punctuation(text):
    doc = nlp(text)
    tokens = [token.lemma_.lower() for token in doc if not token.is_punct and not token.is_stop and not token.like_num]
    tokens = [token for token in tokens if token.strip()]
    return " ".join(tokens)

df_test["text"] = df_test["translated"].apply(tokenize_lemma_and_remove_stopwords_and_punctuation)

In [14]:
df_test.head()

,COUNTRY,Max of REVIEW_YEAR,RISK_DESCRIPTION,Standard_risk_L2,Standard_risk_L1,LIKEHOOD,IMPACT,OVERALL_RISK,RISK_TREATMENT,MITIGATION_MEASURES,COMMON_SERVICE_LINE,COMMON_SERVICE_CATEGORY,COMMON_SERVICE,Language,translated,text
0,Afghanistan,2023,A funding cut imposed by the legislative bodies,,NaN,Possible,Significant Concern,High,Accept,Adjust programme delivery according to the fun...,Common Administration services and including c...,Premises management,Management of common premises,en,A funding cut imposed by the legislative bodies,funding cut impose legislative body
1,Afghanistan,2023,A funding cut imposed by the legislative bodies,,NaN,Possible,Significant Concern,High,Accept,Adjust programme delivery according to the fun...,Common Administration services and including c...,Premises management,Management of common premises,en,A funding cut imposed by the legislative bodies,funding cut impose legislative body
2,Afghanistan,2023,A funding cut imposed by the legislative bodies.,,NaN,Possible,Significant Concern,High,Accept,Adjust programme delivery according to the fun...,Common Administration services and including c...,Premises management,Management of common premises,en,A funding cut imposed by the legislative bodies.,funding cut impose legislative body
3,Afghanistan,2023,A funding cut imposed by the legislative bodies.,,NaN,Possible,Significant Concern,High,Accept,Adjust programme delivery according to the fun...,Common Administration services and including c...,Premises management,Office space and accomodation,en,A funding cut imposed by the legislative bodies.,funding cut impose legislative body
4,Afghanistan,2023,A funding cut imposed by the legislative bodies.,,NaN,Possible,Significant Concern,High,Accept,Adjust service delivery according to the fundi...,Common Administration services and including c...,Transport Services,Vehicle maintenance,en,A funding cut imposed by the legislative bodies.,funding cut impose legislative body


In [15]:
# download the processed file

df_test.to_excel('Service_Risk_preprocessed.xlsx', index=False)

from google.colab import files
files.download('Service_Risk_preprocessed.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>